##  はじめに

### 講義内容
- クラスを利用したプログラミング  
- 塩基/アミノ酸配列データの扱い方 (Biopythonの利用)  

### 対象
- 塩基/アミノ酸配列データやアノテーションデータを取り扱う人
- 初級レベル (Pythonの基本プログラミングは理解できている人)

サンプルデータとして、https://www.ncbi.nlm.nih.gov/assembly/GCF_000146045.2 から取得した出芽酵母 Saccharomyces cerevisiae S288C のゲノム塩基配列（FASTA形式）と遺伝子アノテーションを含んだファイル（GenBank形式およびGFF形式）を用いる。

Biopythonを使用するので、
```
pip install biopython
```
または、condaを使用している場合には
```
conda install biopython
```
でインストールしておく必要がある。

# クラスの利用

## クラスとオブジェクト

- __オブジェクト__  
  Pythonプログラムの中で扱われる文字列や数値、あるいはリストや辞書といったデータ  
  関数もオブジェクトの一つとして扱われる

Pythonプログラムの中で扱われるさまざまな「モノ」に対しての総称が「オブジェクト」といえる。

オブジェクトにはデータの種類に応じて「型」があり、type関数を使用することでオブジェクトがどのような「型」に属するか確認できる。

In [ ]:
type(10)

In [ ]:
my_str = "Hello world"
type(my_str)

In [ ]:
type(print)

- __クラス__  
  オブジェクトの「型」(= オブジェクトの設計図)


オブジェクトの「型」のことを __クラス__ という。  
例えば、整数データはintクラスに属するオブジェクト、文字列データはstrクラスに属するオブジェクト、print関数は builtin_function_or_method クラスに属するオブジェクトといえる。

オブジェクトはデータだけではなく、そのデータに対する様々な処理機能（ __メソッド__ ）を合わせ持っている。
各オブジェクトがどのようなメソッドを持っているかはdir関数を使用で確認できる。

In [ ]:
print(dir(my_str))

strオブジェクトには大文字に変換するためのupperや分割を行うためのsplitといった文字列に対する操作を行うためのメソッドが含まれる。これらを使用するには、変数名の後にピリオドに続けてメソッド名を指定する

In [ ]:
my_str.upper()

In [ ]:
my_str.split(" ")

メソッドを呼び出すにはメソッド名の後の括弧に引数を指定する。  
上記の例で、upperは特に引数を必要としないため括弧内は空欄になっているのに対し、splitの場合には分割に用いる文字を引数に指定している。  
(何も引数を指定せずにsplitを使用することもでき、その場合にはスペースやタブなど空白とみなせる文字が分割の対象となる。)

## 独自のクラスを設計する

より複雑な機能やデータをもったオブジェクトを扱うために、拡張モジュールをインポートして定義済みの型（たとえばdatetimeオブジェクトなど）を利用したり、あるいは自分でオブジェクトの設計図（＝型）である「クラス」を定義したりすることができる。

Biopythonをインポートすれば塩基配列やアミノ酸配列データを扱うためのSeqオブジェクトが利用可能できるが、ここでは自分でクラスを定義する例としてFASTA形式で記述された配列データを格納するためのクラスを設計してみる。

下記は１件のFASTA形式の配列データの例を示す。
```
>gene01 nucleotide sequence of tRNA-Ser
TGGAGTGTTGTCCGAGCGGCTGAAGGAGCATGATTGGAAATCATGTATACGGGTAAATACCTGTATCGAGGGTTCAAATCCCTCACACTCCGT
```
">"で始まる行はタイトル行で、一般に最初の空白までが配列IDやアクセッション番号を示し、それ以降の文字列は遺伝子の機能名や任意の説明書きが含まれる。  
2行目が配列データを示す。この例では改行が含まれていないが、60〜100文字単位で改行が含まれることもある。

FASTAファイルの例 (multi FASTAファイル)

In [ ]:
! head data/s288c.protein.faa

これらの情報を格納するために、  
- 配列ID (id)  
- 配列に対しての説明 (description)  
- 配列自体のデータ (seq）  

といった情報を含んだクラスを設計する。また、この配列データに対しての操作としてGC含量（塩基配列中のGおよびCの割合）を得るためのメソッドを例として定義する。

In [ ]:
class Fasta:
    def __init__(self, id, description, seq):
        self.id = id
        self.description = description
        self.seq = seq
            
    def get_gc_content(self):
        g_count = self.seq.count("G")
        c_count = self.seq.count("C")
        gc_content = ( g_count + c_count ) / len(self.seq)
        return gc_content

最初に定義したメソッド`__init__`は、オブジェクトの設計図であるクラスに実際のデータを格納してオブジェクトを生成する際に実行されるコンストラクタと呼ばれる特殊メソッド。    
メソッドの第一引数は自身のオブジェクトを指すもので、Pythonでは慣例的に "self" が使われる。残りの３つはオブジェクト生成時に与える引数で、与えられた情報はオブジェクトの内部的な変数である`self.id`、`self.description`、`self.seq`にそれぞれ格納されるように定義している。


実際にデータを与えて配列データを格納したオブジェクトを作成してみる。引数"self"は内部的に使用されるだけなので、オブジェクト生成時には残りの3つの引数を与える。

In [ ]:
fasta = Fasta("gene01", "nucleotide sequence of tRNA-Ser", 
              "TGGAGTGTTGTCCGAGCGGCTGAAGGAGCATGATTGGAAATCATGTATACGGGTAAATACCTGTATCGAGGGTTCAAATCCCTCACACTCCGT")

これで先ほどの１件のFASTA形式のデータを持ったオブジェクトを生成し、fastaという変数に格納できたことになる。  
設計図であるクラスに対し、このように実際のデータが格納されてできたオブジェクトの実体を __インスタンス__ と呼ぶ。  
Pythonでは慣例的にクラスの名称には大文字で始まる名前を用い、インスタンスに対しては小文字を使用することが推奨されている。

![fig2](img/fig1_class.png)

内部的な変数（インスタンス変数）にアクセスするには次のようにする。

In [ ]:
fasta.description

クラスの定義時には"self"という変数を使用したが、これは定義時に用いた仮のものなので実際のデータにアクセスするにはインスタンスを示す変数の後ろにピリオドをつけて指定する。通常の変数と同じように新たな値を代入することも可能。

配列データ自体は通常の文字列と同じ扱いなので、次のように部分配列を取り出すこともできる。

In [ ]:
fasta.seq[:10]

生成されたfastaがどのようなクラスに属するか確認をしてみる。

In [ ]:
type(fasta)

このように独自に定義されたクラスFastaに属するオブジェクトであることがわかる。

最後にこの配列のGC含量を求めてみる。インスタンス変数にアクセスしたときと同様にピリオドに続けてメソッド名を指定する。

In [ ]:
fasta.get_gc_content()

get_gc_contentメソッドに定義されている引数は"self"だけで、これは外部から指定しなくても自動的に自分自身を指す変数として扱われるため特に引数を与える必要はない。そのため、この例では括弧内は空欄となっている。


## Dataclassの利用

Python3.7以降では、データを格納するためのクラス Dataclass が利用可能になっている。これを用いるとクラス定義をよりシンプルに書ける。
Dataclass を利用するには、モジュールをインポートする必要がある。

In [ ]:
from dataclasses import dataclass

Dataclass を使うと先のFASTAデータを扱うためのクラスは以下のように書ける。

In [ ]:
@dataclass
class Fasta:
    id: str
    description: str
    seq: str
    
    def get_gc_content(self):
        g_count = self.seq.count("G")
        c_count = self.seq.count("C")
        gc_content = ( g_count + c_count ) / len(self.seq)
        return gc_content

Dataclassを使うとコンストラクタ (\_\_init\_\_メソッド) の記載は不要で、変数名とその型名を列挙するだけでクラスを定義できる。

使い方は通常のクラスと変わらない。

In [ ]:
fasta2 = Fasta("gene01", "nucleotide sequence of tRNA-Ser", 
              "TGGAGTGTTGTCCGAGCGGCTGAAGGAGCATGATTGGAAATCATGTATACGGGTAAATACCTGTATCGAGGGTTCAAATCCCTCACACTCCGT")

print関数を使うとオブジェクトの内容が分かるように表示される。

In [ ]:
print(fasta2)

通常のクラスから生成されたオブジェクトの場合は以下のようになる。

In [ ]:
print(fasta)

## 実践的なクラスの利用

先に定義した独自クラスFastaを利用した実践的なプログラム例として、複数のFASTA形式のデータを含んだファイル（multi FASTA）からデータを１件ずつ取り出しそれぞれの配列の配列IDや長さ、GC含量等を出力するプログラムを作成する。

はじめにmulti FASTAファイルから１件ずつFastaオブジェクトとしてデータを取り出す関数を作成する。

１行ずつファイルを読み込んでいき、`>` で始まるタイトル行が現れたらIDとdescription情報を取得、それ以外の行では配列情報を取得して編すに格納していく。  
１件のデータを読み終わった時点でそれまで格納されていた情報をFastaオブジェクトにして返す。

![fig1](img/fig2_fasta.png)

注意点:  
ファイルの先頭を読み込んだ時点では配列データが得られていないので処理を分ける必要がある。  
ファイルを最後の行まで読み込んだ後で、一番最後の配列データを取り出す処理を行う。



In [ ]:
def read_fasta(file_name):
    with open(file_name) as fh:
        line = next(fh)  # next関数を使い1行目（タイトル行）だけ読み込む。
        seq_id, description = line.strip(">\n").split(" ", 1)  # タイトル行を分割し始めの空白までを配列ID、空白以降をdescriptionとする。空白が含まれていない場合エラーになるので注意。
        seq = ""  # 配列情報を格納する変数を初期化
        for line in fh:  # 2行目以降の読み込み
            if line.startswith(">"):  # ">"で始まる行が現れたらその時点までの配列データを返す
                yield Fasta(seq_id, description, seq)
                seq_id, description = line.strip().strip(">").split(" ", 1)  # 新たなタイトル情報を格納する
                seq = ""
            else:
                seq += line.strip("\n").upper()  # ">"で始まらない場合には、配列データを読み込んで追加していく。upperを加えたのは大文字に変換するため。
        yield Fasta(seq_id, description, seq)  # ループ終了時に最後の配列データを返す
                

__\[参考\]__  
ここで定義した関数 read_fasta は通常の関数とは異なり __ジェネレーター関数__ と呼ばれる。  
通常の関数では return 文が現れた時点で処理を終了して値を返すのに対し、ジェネレーター関数では yield 文を用いて値を返すという違いがある。値を返した時点でジェネレーターは一旦処理を停止してその時点での情報を内部に保ち、再度呼び出しがあると再びyield文が現れる時点まで処理を行って新たな値を返す。  
ジェネレータはリストと同じように for ループと組み合わせることで１件ずつデータを取り出すことができるが、リストがすべてのデータをメモリ上に保持しているのに対して、ジェネレーターは１件ずつのデータしか保持していない点で違いがある。次世代シークエンサーデータのような巨大なデータを扱う際に、１件ずつデータを取り出して処理を行うのに役立つ。


以下が実際の使用例で、配列１件ずつを取り出して配列ID、配列の説明、長さ、GC含量をタブ区切りの文字列として出力している。


In [ ]:
for fasta in read_fasta("data/s288c.fna"):
    print(f"{fasta.id}\t{fasta.description}\tLength={len(fasta.seq):,d}\tG+C%={fasta.get_gc_content():.1%}")

__\[参考]__  
`f`を先頭につけた文字列は"f文字列"と呼ばれるPython3.6から導入された文字列を整形するのに便利な形式で、{ }で括った中に変数名を記載することで変数の中身を文字列中に埋め込むことができる。  
また、変数名の後に`:`で区切って各種書式を指定することも可能で、上記の例では`:,d`を付けて3桁ごとにカンマで数値の桁数を区切ったり、`:.1%`と付けることで小数値を百分率表記にして小数点以下の桁数を指定したりすることができる。



## ここまでのまとめ

- 独自のクラスを定義しオブジェクトにデータを格納することで、データやそれに付随した操作をひとまとめの変数として扱うことができる。  
- クラスに様々なメソッドを加えることでより多様な処理を行えるように拡張できる (塩基配列のデータであれば相補鎖配列に変換するメソッドや、アミノ酸配列への翻訳を行うメソッドなど)。  
- クラスの利用例として、FASTA形式のデータを格納するためのクラス定義とmulti FASTA形式のファイルから１件ずつデータを取り出す例を紹介した。  
    データを格納するための独自クラスを定義してファイルを読み込むための関数と組み合わせて処理を行う方法は、様々な場面に応用できる。  


ここまではクラスの使い方について学ぶためにFasta形式のデータを扱うクラスを定義したが、以下に紹介するBiopythonを用いれば自分でクラス定義を行うことなく配列データを簡単に扱うことができる。

## より高度なクラスの利用 (補足)

### 特殊メソッドやスタティックメソッド

- 特殊メソッド  
  `__`で囲まれたメソッドは特殊メソッドで、通常は直接呼び出されて使うことはなく、他の関数にオブジェクトが引数として渡された場合の挙動を定義する場合などに使われる。  
  例
  - `__len__`  
    len関数に引数として渡された場合の処理を定義できる。下の例では配列の長さを返すようにしている。
  - `__repr__`  
    オブジェクトに格納された情報の概要をprint関数やrepr関数で表示させるときの形式を定義するためのもので、おもにデバッグ用途に用いる。
    Dataclassを使った場合には自動で定義されているが、自分で定義を上書きすることもできる。デフォルトだと配列情報が全て表示されてしまい画面から溢れてしまうので、ここではID、description、長さのみを表示させるように定義し直している。

  
- スタティックメソッド (staticmethod)  
  そのクラスには関係しているが、インスタンスを作ることなく使用できるメソッドで (クラスに属さない) 通常の関数と同じように使える。下記で定義した `read_fasta` はFASTAファイルを読み込んでFastaオブジェクトを返すジェネレータ関数である。スタティックメソッドはインスタンス変数やメソッドにアクセスすることはできず、そのため引数にも "self" は不要である。

In [ ]:
@dataclass
class Fasta:
    id: str
    description: str
    seq: str
    
    def get_gc_content(self):
        g_count = self.seq.count("G")
        c_count = self.seq.count("C")
        gc_content = ( g_count + c_count ) / len(self.seq)
        return gc_content
    
    def __len__(self):
        """len関数に引数として与えた場合に配列の長さを返すように定義"""
        return len(self.seq)
    
    def __repr__(self):
        """print関数に引数として与えた場合に表示される文字列を定義"""
        return f"<Fasta: {self.id}, {self.description}, Length={len(self)}>"
    
    @staticmethod
    def read_fasta(file_name):
        def _parse_title(title):
            title = title.strip(">\n ")
            if " " in title:
                seq_id, description = title.split(" ", 1)
            else:
                seq_id, description = title, ""
            return seq_id, description
        
        with open(file_name) as fh:
            line = next(fh)
            seq_id, description = _parse_title(line)
            seq = ""
            for line in fh:
                if line.startswith(">"):
                    yield Fasta(seq_id, description, seq)
                    seq_id, description = _parse_title(line)
                    seq = ""
                else:
                    seq += line.strip("\n").upper()
            yield Fasta(seq_id, description, seq)

In [ ]:
# FASTAファイルを読み込み、各配列をFastaオブジェクトを要素とするリストに格納する)
fasta_list = list(Fasta.read_fasta("data/s288c.fna"))

In [ ]:
# 配列を長いものから順に並び替える
# 並び替えのキーとしてlen関数を与えているので、__len__メソッドで返される値によってソートされる
fasta_list = sorted(fasta_list, key=len, reverse=True)

In [ ]:
# 配列情報の表示
# pabsrint時には __repr__で定義した情報が表示される。
for fasta in fasta_list:
    print(fasta)

---

# Biopythonの使い方

Biopythonは生命科学に関するデータを扱うための拡張ライブラリ。ここではBiopythonを使った配列ファイルの処理方法を扱う。


In [ ]:
# Biopythonのインポート
# SeqIOはFASTA, GenBankなどの配列データの読み書きを行うためのモジュール
from Bio import SeqIO

## FASTAファイルの読み込み

In [ ]:
fasta_file_name = "data/s288c.fna"

In [ ]:
records = SeqIO.parse(fasta_file_name, "fasta")

In [ ]:
# records は イテレータオブジェクト
type(records)

イテレータはfor文を使って１件ずつデータを取り出すことができる。リストは全データをメモリに保持しているのに対し、イテレータは一度に１件ずつしかデータを保持していないという違いがある。`list(records)`とすればリストに変換することもできる。  
ジェネレータはイテレータの一種。


for文を使って順次データを取り出す方法以外に、次のようにnext関数を使用する方法がある。

In [ ]:
# 一件目 (0番目) の配列データを取り出す
r0 = next(records)

In [ ]:
# 配列データは SeqRecord オブジェクトになっている。
# SeqRecord は配列自体の情報の他に、ID, description, アノテーションなども含む。
r0

nextを繰り返し用いると以降の配列を順次取り出すことができる。前のデータに戻ることはできないので、最初に戻るには`SeqIO.parse`を再び行う必要がある。

In [ ]:
print(r0)

In [ ]:
# 配列ID, descriptionなどの確認、長さは len 関数で取得できる。
print(r0.id)
print(r0.description)
print(len(r0))
print(r0.features)  # FASTAファイルなのでアノテーションは含まれていない --> featuresは空

In [ ]:
# 配列を取り出す。
r0_seq = r0.seq

In [ ]:
# 配列はSeq型のオブジェクトになっている。
r0_seq

Seq型オブジェクトは部分配列の切り出し、相補鎖変換 reverse_compliment, 翻訳 translate などの機能を持つ。

In [ ]:
r0_seq[1806:2169]  # 遺伝子コード領域の切り出し。(1807-2169番目の塩基)

配列の切り出しはFeatureLocationオブジェクトのextractメソッドを使用する方法もある。  
遺伝子がコードされているストランドの切り出しや、複数エクソンから構成される遺伝子配列の切り出しにも対応しているので便利。(後で紹介）

In [ ]:
# 相補鎖変換
r0_seq[1806:2169].reverse_complement()

In [ ]:
# 相補鎖変換　→　翻訳
r0_seq[1806:2169].reverse_complement().translate()  

In [ ]:
# GenBank形式への変換  --> ValueError: missing molecule_type in annotations というエラーが出る
print(r0.format("genbank"))

In [ ]:
# エラーを回避するには、molecule_type を指定する必要がある。
r0.annotations["molecule_type"] = "DNA"
print(r0.format("genbank")[:1000])  # 長いので先頭の1000文字だけ表示

__\[参考\]__ 以前のBiopythonでは ValueError: Need a Nucleotide or Protein alphabet というエラーが出た。  
その場以下のようにしてエラーを回避できる。
```
from Bio.Alphabet import generic_dna
records = SeqIO.parse(fasta_file_name, "fasta", generic_dna)
r0 = next(records)
print(r0.format("genbank")[:1000])  # 長いので先頭の1000文字だけ表示
```

## すべての配列をループで回す

In [ ]:
# recordsはジェネレータなので、リストと同じようにforループで回すことができます。
records = SeqIO.parse(fasta_file_name, "fasta")
for r in records:
    print("Seq ID=", r.id)
    print("Length=", len(r))
    print(r.seq[:50] + "...")
    print("----")

In [ ]:
# list関数を使うことでも同様のことが可能。(ファイル全体をメモリに読み込むので大きいファイルの扱いには注意)
records = list(SeqIO.parse(fasta_file_name, "fasta"))

In [ ]:
# リストにした場合には、indexを指定してデータを抽出することが可能。例えば、最後の配列を取り出すには-1を指定する。
records[-1]

__練習__  
s288c株の遺伝子タンパク質配列FASTAファイル（s288c.protein.faa）を読み込み、閾値未満の長さの配列を取り除き、降順（長いもの順）に
出力するスクリプトを作成せよ。  
下記のテンプレートを使うこと。下にヒントあり。

In [ ]:
protein_fasta_file = "data/s288c.protein.faa"
records = list(SeqIO.parse(protein_fasta_file, "fasta")) 
threshold = 1000

# 閾値未満のものを除く処理を追加
# recordsを降順にソートする

i = 0
for r in records:
    print(r)
    print("Length=", len(r))
    print("-----")
    i += 1
    if i == 5:
        break  # 5件出力した時点で処理を停止する。(画面が溢れるのを防ぐため)

__ヒント1__  
リスト内包表記を使ったフィルタリング

In [ ]:
# リスト内包表記を使うとリストから新しいリストを生成できる。
L = [1, 2, 3, 4, 5]
[x*x for x in L]

In [ ]:
# 値が3より大きい要素を抽出したリストを生成する
L = [1, 2, 3, 4, 5]
[x for x in L if x>3]

__ヒント２__  
keyを指定して様々な方法でソートを行う

In [ ]:
# 文字列の長さでソート
L = ["cat",  "horse", "buffalo", "dog", "ox", "hippopotamus", "tiger"]
sorted(L, key=len, reverse=True)


keyには"大小"や"順番"が判定できる値を返す任意の関数を指定することができる。自分で定義した関数を指定することもできる。  
下記はlambda式（無名関数）を使った例

In [ ]:
# lambda式を使う例。geneの後の数字部分でソート
L = ["gene_1", "gene_10", "gene_2", "gene_21", "gene_101", ]
print("普通にソート", sorted(L))
print("数字部分でソート", sorted(L, key=lambda x: int(x.split("_")[1])))

## ファイルを読み込み、辞書として格納

__本講義では詳細は割愛__

SeqIO.parseはFASTAファイルに含まれる配列データを前から順にしか扱うことができない。配列データを辞書に格納すると任意の配列を参照することができる (ランダムアクセス)

In [ ]:
# SeqIO.to_dict関数を使うと、配列IDをkeyとした辞書にデータを格納することができる。
records = SeqIO.parse(fasta_file_name, "fasta")
dict_records = SeqIO.to_dict(records)

In [ ]:
# 全17件の辞書ができる
len(dict_records)

In [ ]:
dict_records.keys()

In [ ]:
dict_records["NC_001224.1"]

上記の方法はファイルに含まれるすべての配列情報をメモリに格納するので、大きなファイルの取り扱いには向いていない。  
別の方法としてSeqIO.index関数を使う方法もある。

In [ ]:
idx_records = SeqIO.index(fasta_file_name, "fasta")

In [ ]:
# idx_recordsはSeq
type(idx_records)

In [ ]:
# idx_recordsは辞書と同じように使用できる。
# to_dictと異なりメモリ上にデータを保持していないため、処理速度は若干遅くなる。
idx_records.get("NC_001224.1")

In [ ]:
print(idx_records.get_raw("NC_001224.1").decode()[:300])  # get_rawで元の形式のまま取り出せる(長いので先頭300文字のみ表示しています)

# GenBankファイルの読み込み`
FASTAファイルと同様に SeqIO.parse を利用して読み込み可能。`format="genbank"`を指定する。

### GenBank形式ファイル

- __NCBI GenBank の公開形式ファイル (flatfile)__
    - A. ヘッダー部分  
      生物種情報、登録者情報、文献情報、登録日などの情報が含まれる  
    - B. アノテーション部分  
      配列に対する生物学的記述 (biological feature)  
      それぞれの biological feature は位置情報 (location) と、さまざまな属性値 (qualifier) から構成される  
      例) 遺伝子領域やリピート領域などの記載  
    - C. 配列部分  


![fig2](img/fig_genbank_format.png)

図) 羊土社・実験医学別冊「独習 Pythonバイオ情報解析」より引用

## アノテーション情報の確認

FASTAファイルの場合と異なり、GenBank形式には配列に対する様々な注釈情報 (アノテーション) が含まれる。  
Biopythonを利用すればこれらを簡単に取得することができる。

In [ ]:
gbk_file_name = "data/s288c.gbk"

In [ ]:
# ここではlist関数を利用して、全データをリストとして読み込んでいる。
records = list(SeqIO.parse(gbk_file_name, "genbank"))

In [ ]:
# １件目 (0番目)
r0 = records[0]

In [ ]:
print(dir(r0))

In [ ]:
# FASTAファイルを読み込んだときには空だったannotationsに配列のメタデータが辞書として格納されている。
# ここでの "annotation" は配列全体に対する生物種情報や登録者情報といったメタデータのことを示す。
# 遺伝子領域などの生物学的注釈情報は後述のfeaturesに含まれている。
r0.annotations

In [ ]:
# アノテーションされた遺伝子領域の情報はfeaturesにリストとして格納されている。(先頭10件のみ表示)
r0.features[:10]

In [ ]:
# 先頭(0番目)のfeatureを取り出す
f0 = r0.features[0]

In [ ]:
# 先頭のfeatureはsource feature
# source featureはその配列の由来について記述したもので、通常は配列1本について先頭に1件のみ記載される。
print(f0)

In [ ]:
# 3-5番目のfeature (gene) を取り出す
f3, f4, f5 = r0.features[3:6]

In [ ]:
# featureのおもな構成要素は、type, location, qualifiers 
# ここでは、gene, mRNA, CDS に同じ、locus_tag　が割り当てられていることから、これらが対応関係にあることがわかる。
print(f3)
print(f4)
print(f5)

下記がGenBankファイル中での５番目のfeatureについての記載
```
     CDS             complement(1807..2169)
                     /gene="PAU8"
                     /locus_tag="YAL068C"
                     /note="hypothetical protein; member of the seripauperin
                     multigene family encoded mainly in subtelomeric regions"
                     /codon_start=1
                     /product="seripauperin PAU8"
                     /protein_id="NP_009332.1"
                     /db_xref="GeneID:851229"
                     /db_xref="SGD:S000002142"
                     /translation="MVKLTSIAAGVAAIAATASATTTLAQSDERVNLVELGVYVSDIR
                     AHLAQYYMFQAAHPTETYPVEVAEAVFNYGDFTTMLTGIAPDQVTRMITGVPWYSSRL
                     KPAISSALSKDGIYTIAN"
```


以下に、上記の記載情報を取り出す方法を説明する。

In [ ]:
# qualifiersは辞書（順番情報を保持したOrderedDict）になっている。
f5.qualifiers

In [ ]:
# db_xref情報を取り出す。（関連するDBへの参照情報）
# 2件の情報がリストとして格納されていることがわかる。
f5.qualifiers["db_xref"]

In [ ]:
# 遺伝子産物名 (product) を確認してみる。
f5.qualifiers["product"]

In [ ]:
# 上記結果はリストになっているので文字列として取り出すには f5.qualifiers["product"][0] などとする必要がある。
f5.qualifiers["product"][0]

In [ ]:
# 上記の方法だと "product" がqaualifierに存在しない場合エラーになる。getを使うと存在しなかった場合には第二引数に指定した値が返されるのでエラーが発生しない。
f5.qualifiers.get("product", [""])[0]

In [ ]:
# note に情報を追加する。(リストなのでappendで値を追加する)
f5.qualifiers["note"].append("Test")
print(f5)

In [ ]:
# locationを確認してみる。
# 0を起点としているので、開始位置1806は実際には1807番目の塩基を指す。
f5.location

In [ ]:
# locationオブジェクトを利用して配列の切り出しができる。
# locationのstrandが -1 の場合には、自動的に相補鎖側が切り出される。
# extractメソッドの引数には SeqRecord または Seq オブジェクトどちらを与えてもOK。
f5.location.extract(r0.seq)

In [ ]:
# 配列を切り出して翻訳したものを qualifier の "translation" に記載された値と比較してみる。
print(str(f5.location.extract(r0.seq).translate()))
print(f5.qualifiers["translation"][0])

#### [TIPS\] 複数のエクソンから構成される遺伝子の切り出しにも対応している。

130番目のfeatureは2つのエクソンから構成されるCDSとなっている。
```
     CDS             join(87286..87387,87501..87752)
                     /gene="SNC1"
                     /locus_tag="YAL030W"
                     /note="Vesicle membrane receptor protein (v-SNARE);
                     involved in the fusion between Golgi-derived secretory
                     vesicles with the plasma membrane; proposed to be involved
                     in endocytosis; member of the synaptobrevin/VAMP family of
                     R-type v-SNARE proteins; SNC1 has a paralog, SNC2, that
                     arose from the whole genome duplication"
                     /codon_start=1
                     /product="SNAP receptor SNC1"
                     /protein_id="NP_009372.1"
                     /db_xref="GeneID:851203"
                     /db_xref="SGD:S000000028"
                     /translation="MSSSTPFDPYALSEHDEERPQNVQSKSRTAELQAEIDDTVGIMR
                     DNINKVAERGERLTSIEDKADNLAVSAQGFKRGANRVRKAMWYKDLKMKMCLALVIII
                     LLVVIIVPIAVHFSR"
```

In [ ]:
f130 = r0.features[130]

下記のようにエクソン領域を結合して切り出せる。(下記例では翻訳まで行っている)

In [ ]:
f130.location.extract(r0.seq).translate()

#### [TIPS] SeqRecord オブジェクトを切り出すと、その領域中のアノテーションも含めて切り出せる。

In [ ]:
region_f3 = f3.location.extract(r0)
region_f3

抽出した領域に含まれる feature 情報も引き継がれ、このとき各 feature の位置情報は抽出した領域に合わせて調整される。

In [ ]:
region_f3.features

SeqRecordをスライスした場合もアノテーションも含めて切り出される。

In [ ]:
# ↑に示した1a30番目のfeatureを含んだ領域
region_r130 = r0[87285:87752]

In [ ]:
region_r130.features

## ファイル全体の feature をループで回す

record (entry) --> feature --> qualifier の階層構造になっていることを意識すると良い。

例としてCDSフィーチャーの中から全てのアミノ酸配列を辞書として取得する処理を実装する。

In [ ]:
# すべてのアミノ酸配列を辞書として取得 (translationを含まないCDSがあったため失敗した)
D = {}
for record in SeqIO.parse(gbk_file_name, "genbank"):
    for feature in record.features:
        if feature.type == "CDS":
            locus_tag = feature.qualifiers["locus_tag"][0]
            translation = feature.qualifiers["translation"][0]
            D[locus_tag] = translation

In [ ]:
# すべてのアミノ酸配列を辞書として取得 ver2 (try-exceptでエラーを補足し、原因を確認)
D = {}
for record in SeqIO.parse(gbk_file_name, "genbank"):
    for feature in record.features:
        if feature.type == "CDS":
            try:
                locus_tag = feature.qualifiers["locus_tag"][0]
                translation = feature.qualifiers["translation"][0]
                D[locus_tag] = translation
            except KeyError as e:
                # エラーが起こった場合の処理
                print(feature) # 問題のあったfeatureを表示
                raise e  # 再度、エラーを生じさせて処理を停止させる。


↑ qualifierに "pseudo" という記載がある場合には、translation が存在しないことがエラーの原因であった。  
そこで、"pseudo" の場合には処理を行わないように変更する。

In [ ]:
# すべてのアミノ酸配列を辞書として取得 ver3 (完成版)
D = {}
for record in SeqIO.parse(gbk_file_name, "genbank"):
    for feature in record.features:
        if feature.type == "CDS":
            if "pseudo" in feature.qualifiers:
                continue
            try:
                locus_tag = feature.qualifiers["locus_tag"][0]
                translation = feature.qualifiers["translation"][0]
                D[locus_tag] = translation
            except KeyError as e:
                print(feature.qualifiers)
                raise e


In [ ]:
D

In [ ]:
# すべてのアミノ酸配列を辞書として取得 ver4 (改良版)
# get 関数を使用してkeyが存在しない場合にも対応
D = {}
for record in SeqIO.parse(gbk_file_name, "genbank"):
    for feature in record.features:
        locus_tag = feature.qualifiers.get("locus_tag", [""])[0]  # get 関数を使い locus_tag qualifierが存在しない場合には空文字 ("") を返す
        translation = feature.qualifiers.get("translation", [""])[0]
        if locus_tag and translation: # locus_tagおよびtranslationのいずれかが空文字列のときには処理しない
            D[locus_tag] = translation

In [ ]:
# getについての補足
test_dict = {1:100, 2:200, 3:300}
print(test_dict.get(1))  # 100が返る
print(test_dict.get(4))  # keyが存在しない場合、None
print(test_dict.get(4, 400))  # 第２引数としてkeyが存在しない場合に返る値を指定可能。400が返る。

**[練習]**  
GenBankファイルの中身をすべてforループで辿り、CDSの塩基配列・ローカスタグ(locus_tag)・遺伝子産物名(product)を取得し、  
下記のような FASTA 形式で出力するスクリプトを完成させよ。  
ただし product については空欄のものがあれば "unknown protein" とすること（getを使う）。  
塩基配列は `location.extract` を使って取得できる。  
__出力例__

```
>YAL068C seripauperin PAU8
ATGGTCAAATTAACTTCAATCGCCGCTGGTGTCGCTGCCATCGCTGCTACTGCTTCTGCAACCACCACTCTAGCTCAATCTGACGAAAGAGTCAACTTGGTGGAATTGGGTGTCTACGTCTCTGATATCAGAGCTCACTTAGCCCAATACTACATGTTCCAAGCCGCCCACCCAACTGAAACCTACCCAGTCGAAGTTGCTGAAGCCGTTTTCAACTACGGTGACTTCACCACCATGTTGACCGGTATTGCTCCAGACCAAGTGACCAGAATGATCACCGGTGTTCCATGGTACTCCAGCAGATTAAAGCCAGCCATCTCCAGTGCTCTATCCAAGGACGGTATCTACACTATCGCAAACTAG
>YAL067W-A hypothetical protein
ATGCCAATTATAGGGGTGCCGAGGTGCCTTATAAAACCCTTTTCTGTGCCTGTGACATTTCCTTTTTCGGTCAAAAAGAATATCCGAATTTTAGATTTGGACCCTCGTACAGAAGCTTATTGTCTAAGCCTGAATTCAGTCTGCTTTAAACGGCTTCCGCGGAGGAAATATTTCCATCTCTTGAATTCGTACAACATTAAACGTGTGTTGGGAGTCGTATACTGTTAG
>YAL067C putative permease SEO1
ATGTATTCAATTGTTAAAGAGATTATTGTAGATCCTTACAAAAGACTAAAATGGGGTTTTATTCCAGTAAAGCGGCAGGTGGAAGACCTGCCAGATGACTTAAATTCAACAGAAATTGTCACTATCTCCAACAGTA...
```


In [ ]:
i = 0
for record in SeqIO.parse(gbk_file_name, "genbank"):
    for feature in record.features:
        if feature.type != "CDS":
            continue  # CDSでない場合には処理をスキップ    
        locus_tag = feature.qualifiers.get("locus_tag", [""])[0]
        ### productを取得する処理を追加
        ### 塩基配列を取得する処理を追加
        i += 1
        ### print で出力する処理を追加
        if i == 5:  # 動作確認のため5件のみ出力したところで処理を停止
            break
    break # 動作確認のため最初の配列 (レコード) のみ処理するようにしている。この行を削除すれば全レコードを対象に処理される。

# GFFファイルの読み込み

__本項は講義では扱いません__

以下のコードの実行にはBCBioGFFが必要。
```
pip install bcbio-gff
```
または、condaを使用している場合には
```
conda install -c bioconda bcbiogff
```
でインストールする。2022年12月現在、pipの場合には ver.0.6.9が、condaの場合には ver.0.6.6 がインストールされた。

現状では Biopython 単独ではGFFファイルの読み書きはできないため、BCBioGFFを利用する。  
GFF ファイルは GenBank ファイルと同等の情報を保持できるが、ID-Parent を対応づけることによって明示的に gene-mRNA-CDS の階層構造を表現している。[参考](https://github.com/The-Sequence-Ontology/Specifications/blob/master/gff3.md)

In [ ]:
# BCBioのインポート
# GFFを扱うためのモジュール (現時点ではBiopythonには含まれていないため、別にインストールする必要がある conda install -c bioconda bcbiogff または pip install bcbio-gff）
from BCBio import GFF

In [ ]:
# ここで使うGFFファイルはRNAseq解析用に遺伝子ID(gene_id)情報を付け足している。
gff_file_name = "data/s288c_e.gff"

In [ ]:
records = GFF.parse(open(gff_file_name))

In [ ]:
# 最初の配列の先頭10件のfeatureを表示
# GenBankファイルの時とは異なり、mRNAやCDS featureが見えない。
# GFFファイルでは mRNA は gene の子フィーチャー "sub_features" という扱いになっているため。
r0 = next(records)
r0.features[:10]

In [ ]:
# 最初の gene feature を取得
gene = r0.features[4]

In [ ]:
# sub_featuresの中にmRNA featureが含まれている。
# この例ではmRNAは1件のみですが、一つの gene (遺伝子座) に対して複数の mRNA が存在する場合もある。
gene.sub_features

In [ ]:
# mRNA featureを取得
mrna = gene.sub_features[0]

In [ ]:
# mRNAのsub_featuresを確認
# CDS featureとexon featureが含まれている。
# sub_featureの種類はファイルによって異なる。CDSのみの場合や、非翻訳領域 (UTR) が含まれている場合もある。
mrna.sub_features

In [ ]:
# sub_features の階層構造をたどり、どのようなfeatureが含まれているかを確認してみる。。
S=set()
for record in GFF.parse(open(gff_file_name)):
    if record.id == "NC_001224.1":
        continue  # ミトコンドリア配列は除外
    for f in record.features:
        feature_type_1 = f.type
        S.add(feature_type_1)
        for sf in f.sub_features:
            feature_type_2 = feature_type_1 + " -> " + sf.type
            S.add(feature_type_2)
            for ssf in sf.sub_features:
                feature_type_3 = feature_type_2 + " -> " + ssf.type
                S.add(feature_type_3)

    

In [ ]:
sorted(list(S), key=lambda x:(x.count("->"), x))  # 階層数でソートしたあと、アルファッベット順でソートして表示している

## 単元4で使用するアノテーション一覧ファイルの作成方法

単元４ではマウスの参照ゲノムGRCm39に対してRNAseqリードをマップして得られた結果をサンプルデータとして用いる。  
その際に用いるアノテーション情報の一覧表を作成したスクリプトは以下の通り。  
完成版スクリプトファイルは `make_annotation_table_from_gff.py` の名称で本ディレクトリ内に置いてある。

GFFファイル中の属性値中の `;`や`,`など一部の記号はURLエンコーディングされているので `urllib.parse.unquote` でデコードしている。

```
import gzip
import urllib.parse
from BCBio import GFF

gff_file_name = "Mus_musculus.GRCm39.108.gff3.gz"

for record in GFF.parse(gzip.open(gff_file_name, "rt")):
    for f in record.features:
        if f.type == "gene":
            gene_id = f.qualifiers["ID"][0].replace("gene:", "")
            # Name と description 属性を持たない場合があるので get を使って値を取得する。
            gene_name = f.qualifiers.get("Name", ["NO_NAME"])[0]
            description = urllib.parse.unquote(f.qualifiers.get("description", ["-"])[0])
            for sf in f.sub_features:
                if sf.type == "mRNA":
                    transcript_id = sf.qualifiers["ID"][0].replace("transcript:", "")
                    version = sf.qualifiers["version"][0]
                    print("\t".join([transcript_id + "." + version, gene_id, gene_name, description]))
```

__実行方法__　　

マウスゲノムのGFFファイルは
```
curl -O https://ftp.ensembl.org/pub/release-108/gff3/mus_musculus/Mus_musculus.GRCm39.108.gff3.gz
```
で取得しておく。  
結果は標準出力に出力されるのでファイルにリダイレクトして結果を保存する。
```
python make_annotation_table_from_gff.py > annotation.tsv
```

## BLAST結果ファイルの処理

参考) [Biopython Tutorial and Cookbook](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec129)

BLASTの出力結果例 (`-outfmt 6`)   
各列は左から、query, subject, % identity, alignment length, mismatches, gap opens, query_start, query_end, subject_start, subject_end, evalue, bit score  

```
query_A	gene_0001	38.444	450	254	7	8	440	8	451	1.96e-90	278
query_B	gene_2155	75.309	891	217	2	13	900	13	903	0.0	1405
query_B	gene_1608	21.091	275	192	9	34	290	9	276	3.11e-06	46.2
query_C	gene_1111	67.282	813	262	2	4	816	1	809	0.0	1098
query_C	gene_0984	40.559	752	425	8	7	749	2	740	0.0	555
query_C	gene_0984	24.336	226	146	6	593	804	481	695	1.03e-07	51.6
```

BiopythonでBLASTの実行結果を処理するためにはXML形式で結果を出力する必要がある (`-outfmt 7`)。  
下記は↑で挙げた結果例をXML形式ファイルで出力したもの。

In [ ]:
! head -100 data/blast.xml

XML形式ファイルは↓の模式図で示したような階層構造になっている。  
Biopythonで結果を処理する場合にもこの構造を模したオブジェクトにデータが格納される。

![fig2](img/fig_blast.png)

BLASTの結果を処理するために NCBIXML モジュールをインポートする。

In [ ]:
from Bio.Blast import NCBIXML

In [ ]:
blast_xml_file = "data/blast.xml"

BLASTオブジェクト - Alignment オブジェクト - HSP オブジェクトという階層構造になっているので for ループで結果を全て辿る。

In [ ]:
blast_records = NCBIXML.parse(open(blast_xml_file))
for blast_record in blast_records:
    print(f"Query: {blast_record.query} (Length={blast_record.query_length})")
    print("\t" + "-" * 40)
    for alignment in blast_record.alignments:
        print(f"\tSubject: {alignment.hit_id}, Definition: {alignment.hit_def} (Length={alignment.length})") 
        for i, hsp in enumerate(alignment.hsps, 1):
            print(f"\t\tHSP{i}: QueryHit={hsp.query_start}-{hsp.query_end} SubjectHit={hsp.sbjct_start}-{hsp.sbjct_end} " +
                  f"Score={hsp.bits} e-value={hsp.expect}")

    print("=" * 80)

AlignmentオブジェクトやHSPオブジェクトがどのような情報を保持しているかは、Biopythonの[APIドキュメント](https://biopython.org/docs/latest/api/Bio.Blast.Record.html) を参照。`dir(alignment)` や `dir(HSP)` などで調べても良い。